# 2. PyTorch dataset and data loader

In the previous notebook, we have learned how to investigate our dataset. Next up, we'll want to create a PyTorch `Dataset` and a PyTorch `DataLoader`. These will help us get our data ready to be passed through a neural network.

## 2.1 Creating a PyTorch dataset

A [PyTorch `Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) is an abstract representation of a dataset. You can do two things with a PyTorch `Dataset`: get the *item* at a certain **index**, and get the **length** of the dataset (i.e., the number of *items*). That's all. What an *item* is, is entirely up to you, but in the case of classification, it is typically a tuple of an image and a class label.

### 2.1.1 Built-in datasets

Torchvision already includes [many datasets](https://pytorch.org/vision/stable/datasets.html) to play around with. Let's take a look at the [MNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST) dataset.

In [1]:
from torchvision.datasets import MNIST

mnist_ds = MNIST("../data/",train=True,download=True)

100%|███████████████████████████| 9912422/9912422 [00:00<00:00, 16729550.96it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|███████████████████████████████| 28881/28881 [00:00<00:00, 40231050.76it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|███████████████████████████| 1648877/1648877 [00:00<00:00, 11674104.72it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 8554346.10it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



We can ask an item at a certain index:

In [2]:
mnist_ds[0]

(<PIL.Image.Image image mode=L size=28x28>, 5)

This gives us two things: an image and an integer label.

In [3]:
img , label = mnist_ds[0]

In [4]:
img

In [5]:
label

5

We can also ask the length of the dataset.

In [6]:
len(mnist_ds)

60000

### 2.1.2 A first attempt to a custom dataset

To implement your own dataset, you need create a class that inherits from [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) and implements two methods: `__getitem__()` and `__len__()`.

For our Pokémon use case:

- `__getitem__()` should return the image at the given index, along with the name of that Pokémon;
- `__len__()` should return the number of images.

In the previous notebook, we saw how we can represent our dataset as a Pandas `DataFrame`. We can use this representation to implement `__getitem__()` and `__len__()` without much new code:

In [7]:
import pandas as pd
from pathlib import Path
from PIL import Image
data_path = Path("../data/Simpsons")
all_files = list(data_path.glob('data/*/*'))
df = pd.DataFrame([
    {

        'image': p,
        'label': p.parent.name
    }
    for p in all_files
])

In [8]:
from torch.utils.data import Dataset

class SimpsonsDataset(Dataset):
    def __init__(self,datapath,transform=None):
        data_path= Path(datapath)
        self.df = pd.DataFrame([
        {
    
            'image': p,
            'label': p.parent.name
        }
        
        
        for p in data_path.glob('data/*/*')
        ])
        self.transform = transform
        
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.df)

We can play around with this dataset, just like with the MNIST dataset.

In [9]:
ds = SimpsonsDataset("../data/Simpsons")
ds[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=330x256>,
 'ned_flanders')

While this is already a perfectly valid PyTorch `Dataset`, there are two issues:

1. Our labels are strings;
2. We don't make a distinction between train, validation and test data.

> 🤔 **Why is it a problem that our labels are strings?**
>
> When training a network, we typically iterate over *batches* of data. These batches are represented as PyTorch tensors. And it is not possible (and not necessary, really) to create a tensor of strings.

We'll solve both issues in the following sections.

### 2.1.3 Replacing string labels with integer labels

First, we'll replace our string labels by integer labels. For this, we'll create a dictionary that maps each string label to a unique integer.

In [10]:
df = pd.DataFrame([
        {
    
            'image': p,
            'label': p.parent.name
        }
        
        
        for p in data_path.glob('data/*/*')
        ])

To get a list of unique labels, we can call [`unique()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) on the `label` column:

In [11]:
df['label'].unique()

array(['ned_flanders', 'bart_simpson', 'otto_mann',
       'apu_nahasapeemapetilon', 'nelson_muntz', 'ralph_wiggum',
       'maggie_simpson', 'carl_carlson', 'homer_simpson',
       'charles_montgomery_burns', 'comic_book_guy', 'martin_prince',
       'edna_krabappel', 'principal_skinner', 'fat_tony', 'chief_wiggum',
       'troy_mcclure', 'sideshow_bob', 'cletus_spuckler', 'agnes_skinner',
       'waylon_smithers', 'barney_gumble', 'rainier_wolfcastle',
       'groundskeeper_willie', 'patty_bouvier', 'milhouse_van_houten',
       'sideshow_mel', 'snake_jailbird', 'lionel_hutz', 'disco_stu',
       'krusty_the_clown', 'gil', 'kent_brockman', 'selma_bouvier',
       'lisa_simpson', 'moe_szyslak', 'miss_hoover', 'lenny_leonard',
       'mayor_quimby', 'abraham_grampa_simpson', 'marge_simpson',
       'professor_john_frink'], dtype=object)

To ensure a consistent mapping across different systems, we can sort these labels with [the built-in Python function `sorted()`](https://docs.python.org/3/library/functions.html#sorted):

In [12]:
sorted(df['label'].unique())

['abraham_grampa_simpson',
 'agnes_skinner',
 'apu_nahasapeemapetilon',
 'barney_gumble',
 'bart_simpson',
 'carl_carlson',
 'charles_montgomery_burns',
 'chief_wiggum',
 'cletus_spuckler',
 'comic_book_guy',
 'disco_stu',
 'edna_krabappel',
 'fat_tony',
 'gil',
 'groundskeeper_willie',
 'homer_simpson',
 'kent_brockman',
 'krusty_the_clown',
 'lenny_leonard',
 'lionel_hutz',
 'lisa_simpson',
 'maggie_simpson',
 'marge_simpson',
 'martin_prince',
 'mayor_quimby',
 'milhouse_van_houten',
 'miss_hoover',
 'moe_szyslak',
 'ned_flanders',
 'nelson_muntz',
 'otto_mann',
 'patty_bouvier',
 'principal_skinner',
 'professor_john_frink',
 'rainier_wolfcastle',
 'ralph_wiggum',
 'selma_bouvier',
 'sideshow_bob',
 'sideshow_mel',
 'snake_jailbird',
 'troy_mcclure',
 'waylon_smithers']

Now, with [the built-in Python function `enumerate()`](https://docs.python.org/3/library/functions.html#enumerate), we can get an iterable that yields another integer value for each of our labels.

In [13]:
for i, label in enumerate(sorted(df['label'].unique())):
    
    print(i, label)

0 abraham_grampa_simpson
1 agnes_skinner
2 apu_nahasapeemapetilon
3 barney_gumble
4 bart_simpson
5 carl_carlson
6 charles_montgomery_burns
7 chief_wiggum
8 cletus_spuckler
9 comic_book_guy
10 disco_stu
11 edna_krabappel
12 fat_tony
13 gil
14 groundskeeper_willie
15 homer_simpson
16 kent_brockman
17 krusty_the_clown
18 lenny_leonard
19 lionel_hutz
20 lisa_simpson
21 maggie_simpson
22 marge_simpson
23 martin_prince
24 mayor_quimby
25 milhouse_van_houten
26 miss_hoover
27 moe_szyslak
28 ned_flanders
29 nelson_muntz
30 otto_mann
31 patty_bouvier
32 principal_skinner
33 professor_john_frink
34 rainier_wolfcastle
35 ralph_wiggum
36 selma_bouvier
37 sideshow_bob
38 sideshow_mel
39 snake_jailbird
40 troy_mcclure
41 waylon_smithers


So, we can create our label-to-integer dictionary with the following `for` loop:

In [14]:
label_to_int = {}
for i, label in enumerate(sorted(df['label'].unique())):
    
    label_to_int[label] = i

Or, we can use a [dictionary comprehension](https://www.geeksforgeeks.org/python-dictionary-comprehension/):

In [15]:
label_to_int = {label: i for i, label in enumerate(sorted(df['label'].unique()))}
label_to_int

{'abraham_grampa_simpson': 0,
 'agnes_skinner': 1,
 'apu_nahasapeemapetilon': 2,
 'barney_gumble': 3,
 'bart_simpson': 4,
 'carl_carlson': 5,
 'charles_montgomery_burns': 6,
 'chief_wiggum': 7,
 'cletus_spuckler': 8,
 'comic_book_guy': 9,
 'disco_stu': 10,
 'edna_krabappel': 11,
 'fat_tony': 12,
 'gil': 13,
 'groundskeeper_willie': 14,
 'homer_simpson': 15,
 'kent_brockman': 16,
 'krusty_the_clown': 17,
 'lenny_leonard': 18,
 'lionel_hutz': 19,
 'lisa_simpson': 20,
 'maggie_simpson': 21,
 'marge_simpson': 22,
 'martin_prince': 23,
 'mayor_quimby': 24,
 'milhouse_van_houten': 25,
 'miss_hoover': 26,
 'moe_szyslak': 27,
 'ned_flanders': 28,
 'nelson_muntz': 29,
 'otto_mann': 30,
 'patty_bouvier': 31,
 'principal_skinner': 32,
 'professor_john_frink': 33,
 'rainier_wolfcastle': 34,
 'ralph_wiggum': 35,
 'selma_bouvier': 36,
 'sideshow_bob': 37,
 'sideshow_mel': 38,
 'snake_jailbird': 39,
 'troy_mcclure': 40,
 'waylon_smithers': 41}

We can plug this into our SimpsonsDataset:

In [16]:
class SimpsonsDataset(Dataset):
    def __init__(self,datapath,transform=None):
        data_path= Path(datapath)
        df = pd.DataFrame([
        {
    
            'image': p,
            'label': p.parent.name
        }
        
        
        for p in data_path.glob('data/*/*')
        ])
        label_to_int = {label: i for i, label in enumerate(sorted(df['label'].unique()))}

        self.df = df
        self.label_to_int = label_to_int
        self.transform = transform
        
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.df)

Let's try out our new dataset.

In [17]:
ds = SimpsonsDataset("../data/Simpsons")
ds[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=330x256>,
 'ned_flanders')

### 2.1.4 Splitting the data into a train, validation and test split

Training a neural network and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would achieve a perfect score but would fail to predict anything useful on data that was not used during training. This situation is called **overfitting**. To get a better sense of model performance on *unseen* data, it is customary to randomly split the dataset into three disjoint subsets.

1. **Training set**: You train the model with the training set.
2. **Validation set**: Every X training iterations, you evaluate the model performance on the validation set. It is common to only keep the model that obtained the best validation score during training. When training multiple models with different configurations (or *hyperparameters*), you should use the evaluation on the validation set to decide which model to keep.
3. **Test set**: *Once you have decided a model based on validation perfomance*, you evaluate on the test set to get an estimate of the model's ability to generalize to unseen data. In many machine learning competitions, to avoid cheating, the test set is either unlabeled or unavailable to the participants.

As you see, the validation set is used to compare different model configurations, or *hyperparameters*. This could simply be the number of iterations used to train the model, but this also includes the choice of data augmentations, optimizer, learning rate, model architecture (number of layers, number of neurons in a layer, kind of layer,...) etc.

To split our dataset into these subsets, we can use [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split).

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
df_train,df_val = train_test_split(df, train_size=0.8, random_state=42)

We can update our `SimpsonsDataset` so that, depending on a given constructor argument, the correct subset is used.

In [20]:
\section{Opdrachtbeschrijving}class SimpsonsDataset(Dataset):
    def __init__(self,datapath, subset, transform=None):
        data_path= Path(datapath)
        df = pd.DataFrame([
        {
    
            'image': p,
            'label': p.parent.name
        }
        
        
        for p in data_path.glob('data/*/*')
        ])
        label_to_int = {label: i for i, label in enumerate(sorted(df['label'].unique()))}

        #Train, test and val
        df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
        df_train, df_val = train_test_split(df, train_size=0.8, random_state=42)

        self.df = df
        self.label_to_int = label_to_int
        self.transform = transform

        if subset == 'train':
            self.df = df_train.reset_index()
        elif subset == 'val':
            self.df = df_val.reset_index()
        elif subset == 'test':
            self.df = df_val.reset_index()
        
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.df)

Let's try it out...

In [21]:
ds_train = SimpsonsDataset("../data/Simpsons",subset='train')
ds_train[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=312x352>,
 'comic_book_guy')

### 2.1.5 Cross-validation

Choosing the model that achieved the best performance on the validation set (instead of the training set) avoids selecting a model that has overfit. However, when comparing many models based on validation score, we might select a model that only worked best for the particular random choice of training and validation data. To make a better-informed decision, we can use **cross-validation** techniques.

A basic cross-validation technique is **$k$-fold cross-validation**. Here, the data set is first split up into only two subsets: a *training* set and a *test* set. The training set is partitioned into **$k$ equally-sized folds**. When training a model, we choose one fold for *validation* and the other $k - 1$ folds for *training*. By training a certain model configuration with all $k$ different choices for training and validation and averaging the validation scores, we can effectively select the best model configuration in a more reliable and robust manner. The figure below illustrates the process of $k$-fold cross-validation ([source](https://scikit-learn.org/stable/modules/cross_validation.html)).

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" style="max-width: 500px; margin: auto; padding: 2em;"/>

To add support for $k$-fold cross validation to our `PokemonDataset`, we'll first use `train_test_split()` to split our dataset into train+val and test set. Then, with [`numpy.array_split()`](https://numpy.org/doc/stable/reference/generated/numpy.array_split.html), we can split the train+val set into $k$ folds.

In [22]:
import numpy as np

df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
k=5
folds =  np.array_split(df_trainval,k) 

/home/jonas/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Now, we can choose which fold to use for validation. The other folds should be concatenated to create the training set.

In [23]:
val_fold = 0
df_val = folds[val_fold]
train_folds = [fold for i, fold in enumerate(folds) if i != val_fold]
df_train = pd.concat(train_folds)

Plugging this into our `SimpsonsDataset`:

In [24]:
class SimpsonsDataset(Dataset):
    def __init__(self,datapath, subset, k=5, val_fold = 0, transform=None):
        data_path= Path(datapath)
        df = pd.DataFrame([
        {
    
            'image': p,
            'label': p.parent.name
        }
        
        
        for p in data_path.glob('data/*/*')
        ])
        label_to_int = {label: i for i, label in enumerate(sorted(df['label'].unique()))}

        #Train, test and val
        df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)

        # Cross Validation
        folds = np.array_split(df_trainval, k)
        df_val = folds[val_fold]
        train_folds = [fold for i, fold in enumerate(folds) if i != val_fold]
        df_train = pd.concat(train_folds)
        
        self.df = df
        self.label_to_int = label_to_int
        self.transform = transform

        if subset == 'train':
            self.df = df_train.reset_index()
        elif subset == 'val':
            self.df = df_val.reset_index()
        elif subset == 'test':
            self.df = df_val.reset_index()
        
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.df)

Now we can create a training set, validation set and test set:

In [25]:
import torch
from torchvision.transforms import v2

# Create transforms
train_tfms = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(30),
    v2.RandomResizedCrop(224, antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Bij de validatie set geen randomness
val_tfms = v2.Compose([
    v2.ToImage(),
    v2.Resize(224, antialias=True),
    v2.CenterCrop(224),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
ds_train = SimpsonsDataset('../data/Simpsons', 'train', transform=train_tfms)
ds_val = SimpsonsDataset('../data/Simpsons', 'val', transform=val_tfms)
ds_test = SimpsonsDataset('../data/Simpsons', 'test', transform=val_tfms)

/home/jonas/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/jonas/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


> ⚠️ **Don't use random transforms for validation or test set!**
>
> To avoid making the validation and test evaluations irreproducible, you should not use randomness in the transforms that you'll use for the validation and test set.

Hooray! 🙌 We now have a full-fledged PyTorch `Dataset` with support for $k$-fold cross-validation! 🎉

## 2.2 PyTorch data loader

A PyTorch `Dataset` allows us to easily get images (and labels) from our dataset. When training a neural network, however, we can save training time by training with **batches** of data, instead of passing the images through the network one-by-one.

Batching your data is the task of the [PyTorch `DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader). Given a PyTorch `Dataset` that returns a tuple with an image tensor and an integer label, it is very simple to create a `DataLoader`:

In [26]:
from torch.utils.data import DataLoader

dl_train = DataLoader(ds_train, batch_size=10)

A `DataLoader` is an iterable, where each iteration item is a batch of data samples. As with any iterable in Python, you can iterate over a `DataLoader` using a `for` loop. Let's inspect what's inside an iteration item.

In [27]:
for x in dl_train:
    break

x

[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -1.8439, -1.8953, -1.9467],
           [-2.1179, -2.1179, -2.1179,  ..., -1.8439, -1.8953, -1.9467],
           [-2.1179, -2.1179, -2.1179,  ..., -1.8439, -1.8953, -1.9467],
           ...,
           [-0.3712, -0.3883, -0.4054,  ..., -1.6042, -1.6042, -1.6042],
           [-0.3712, -0.3883, -0.4054,  ..., -1.4843, -1.5014, -1.5014],
           [-0.3712, -0.3883, -0.4054,  ..., -1.4158, -1.4329, -1.4500]],
 
          [[-2.0357, -2.0357, -2.0357,  ..., -1.8081, -1.8606, -1.9132],
           [-2.0357, -2.0357, -2.0357,  ..., -1.8081, -1.8606, -1.9132],
           [-2.0357, -2.0357, -2.0357,  ..., -1.8081, -1.8606, -1.9132],
           ...,
           [-0.3550, -0.3200, -0.3025,  ..., -1.4405, -1.4405, -1.4405],
           [-0.3375, -0.3200, -0.3025,  ..., -1.3354, -1.3529, -1.3704],
           [-0.3375, -0.3200, -0.3025,  ..., -1.2829, -1.3004, -1.3179]],
 
          [[-1.8044, -1.8044, -1.8044,  ..., -1.2816, -1.3339, -1.3861],
           [-

Our `DataLoader` composes the first batch with the samples at index $0,\ldots, 9$, the second batch $10,\ldots,19$ and so on. To compose the batches with random samples, you can pass `shuffle=True` to the `DataLoader` constructor.

> ⚠️ **Only shuffle the training set!**
>
> It is considered bad practice to shuffle any of your evaluation sets, as this might make your evaluations irreproducible.

In [32]:
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)

When taking a look at the implementation of `__getitem__()` in our `PokemonDataset`, you'll see that it involves loading an image from disk with `read_image()` and applying image transforms. Both of these steps may block the computing process for a while. If we'd sequentially run `__getitem__()` for the 10 indices in our batch, it might take some time before the batch is created. Setting the argument `num_workers` as a positive integer will turn on **multi-process data loading** with the specified number of loader worker processes.

In [33]:
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True, num_workers=10)

/home/jonas/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In summary, the following code creates our train, validation and test datasets and data loaders.

In [34]:
# Create data loaders
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=10, shuffle=False)
dl_test = DataLoader(ds_test, batch_size=10, shuffle=False)

And with this, we have covered everything on the data side! 💪 Our data is now ready to be passed into a neural network!

In [31]:
# ... WRITE YOUR CODE HERE ... #